# Projeto - Natural Language Processing (NLP)

**Integrantes**:
- Antonio Lucas Michelon de Almeida
- Pedro Nery Affonso dos Santos
- Rafael Gordon Paves

In [4]:
import pandas as pd
import re
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
from tqdm import tqdm

In [6]:
# Captura pedido de música
query = input("O que você gostaria de ouvir hoje? ")

## Playlist gerada a partir de uma única requisição ao Gemini (LLM)

In [12]:
def monta_playlist(texto):
    """
    Função que realiza uma requisição de montagem de playlist diretamente para o Gemini.
    """
    
    prompt = f"""
    Você é um especialista em música e construção de playlists. 
    Você deve analisar o seguinte pedido de playlist e construir uma seleção de músicas que se encaixem no tema solicitado.
    Refira-se EXCLUSIVAMENTE ao pedido fornecido para criar a playlist.
    A playlist deve ser composta por 10 músicas, com o nome da música, do artista e possível featuring.
    A resposta deve ser uma lista em formato JSON, com os seguintes campos:
    - nome_musica: Nome da música
    - artista: Nome do artista
    - featuring: Nome do artista convidado (caso exista)

    Pedido do usuário: {texto}
    """
    
    class FormatoResposta(BaseModel):
        nome_musica: str
        artista: str
        featuring: str | None

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': list[FormatoResposta],
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    return resposta

# Resposta do Gemini
resposta = monta_playlist(query)
# Converte a resposta em um dicionário
resposta_dict = json.loads(resposta.text)
# Cria um DataFrame a partir da lista de dicionários
df = pd.DataFrame(resposta_dict)
# Renomeia as colunas
df.rename(columns={'nome_musica': 'Nome da Música', 'artista': 'Artista', 'featuring': 'Featuring'}, inplace=True)
df

,Nome da Música,Artista,Featuring
0,Happy,Pharrell Williams,None
1,Walking on Sunshine,Katrina & The Waves,None
2,September,"Earth, Wind & Fire",None
3,Don't Stop Me Now,Queen,None
4,Good Day,Nappy Roots,None
5,Lovely Day,Bill Withers,None
6,I Want You Back,The Jackson 5,None
7,Hey Ya!,Outkast,None
8,Valerie,Mark Ronson,Amy Winehouse
9,Riptide,Vance Joy,None


## Playlist gerada a partir de interações entre agentes

## Playlist gerada a partir de BERT embeddings